## For this dataset, the goal will be to detect irregular activity or potential fraud.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import math


#se abre el set de datos
df = pd.read_excel('ETL_credit.xlsx', na_values=['na', 'n/a', 'null', 'none', '', '--', '?', 'missing'])

In [24]:
#change the columns's name to upper format
df.columns = df.columns.str.upper()

# delete 'UNNAMED: 0' 
df = df.drop(columns=['UNNAMED: 0'])

#I will use a ".info()" to see if the format of the columns is correct
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2395 entries, 0 to 2394
Data columns (total 24 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   LIMIT_BAL                       2395 non-null   int64 
 1   SEX                             2395 non-null   object
 2   EDUCATION                       2395 non-null   object
 3   MARRIAGE                        2395 non-null   int64 
 4   AGE                             2395 non-null   int64 
 5   PAY_0                           2395 non-null   int64 
 6   PAY_2                           2395 non-null   int64 
 7   PAY_3                           2395 non-null   int64 
 8   PAY_4                           2395 non-null   int64 
 9   PAY_5                           2395 non-null   int64 
 10  PAY_6                           2395 non-null   int64 
 11  BILL_AMT1                       2395 non-null   int64 
 12  BILL_AMT2                       2395 non-null   

Whith this, we can see in this dataframe aren't any missing values, so we can proceed to the next step.

In [25]:
#looking if in the dataframe are any duplicate rows
df.duplicated().sum()

np.int64(0)

In [26]:
df.describe()

,LIMIT_BAL,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,...,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6
count,2395.000000,2395.000000,2395.000000,2395.000000,2395.000000,2395.000000,2395.000000,2395.000000,2395.000000,2395.000000,...,2395.000000,2395.000000,2395.000000,2395.000000,2395.000000,2395.00000,2395.000000,2395.000000,2395.000000,2395.000000
mean,165895.615866,1.580376,35.424635,0.003758,-0.137370,-0.151566,-0.248852,-0.258873,-0.281420,51219.144885,...,45530.633820,41561.627557,40009.523591,38813.497704,5551.259708,5221.94238,4857.470981,4893.472234,4987.059290,5152.078914
std,129486.412237,0.525569,9.451436,1.122967,1.207365,1.247551,1.176504,1.156417,1.181577,77882.887342,...,69641.276168,68126.067839,64335.374111,65477.068756,13704.340652,14619.68646,22160.690090,14254.694001,16899.445034,20080.576190
min,10000.000000,0.000000,21.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-14386.000000,...,-9850.000000,-7905.000000,-28335.000000,-339603.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
25%,50000.000000,1.000000,28.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,3122.000000,...,2309.000000,1864.500000,1529.000000,1018.000000,1000.000000,586.00000,249.500000,244.500000,251.000000,0.000000
50%,140000.000000,2.000000,34.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,21148.000000,...,19476.000000,17919.000000,17596.000000,15829.000000,2120.000000,2000.00000,1443.000000,1500.000000,1500.000000,1329.000000
75%,230000.000000,2.000000,41.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,62995.500000,...,56238.500000,49309.000000,48538.500000,47398.000000,5003.500000,4923.50000,4000.000000,4000.000000,4000.000000,4000.000000
max,1000000.000000,3.000000,75.000000,8.000000,7.000000,7.000000,7.000000,7.000000,8.000000,964511.000000,...,548020.000000,891586.000000,927171.000000,961664.000000,239104.000000,285138.00000,896040.000000,205000.000000,332000.000000,528666.000000


I’m going to create a dataset with the rows that are suspected of irregular activity or fraud by filtering out the rows outside the IQR and adding them to a separate DataFrame for further analysis


In [29]:
def limits (col):
    Q1 = df[f'{col}'].quantile(0.25)
    Q3 = df[f'{col}'].quantile(0.75)
    IQR = Q3 - Q1

    upper_limit = Q3 + 1.5 * IQR
    lower_limit = Q1 - 1.5 * IQR

    return lower_limit, upper_limit

# Select only the numeric columns, excluding the target variable if it is also numeric.
numeric_cols = df.select_dtypes(include='number').columns

# Create a filter to identify outliers in any numeric column.
outlier_mask = pd.Series([False] * len(df))
for col in numeric_cols:
    lower, upper = limits(col)
    outlier_mask = outlier_mask | (df[col] < lower) | (df[col] > upper)

# New DataFrame with all the outliers in at least one column.
df_outliers_all = df[outlier_mask]

df_outliers_all

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,DEFAULT PAYMENT NEXT MONTH (Y)
0,120000,female,university,2,26,-1,2,0,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,default
3,50000,male,university,1,57,-1,0,-1,0,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,not default
5,500000,male,graduate school,2,29,0,0,0,0,0,...,542653,483003,473944,55000,40000,38000,20239,13750,13770,not default
7,140000,female,high school,1,28,0,0,2,0,0,...,12211,11793,3719,3329,0,432,1000,1000,1000,not default
8,20000,male,high school,2,35,-2,-2,-2,-2,-1,...,0,13007,13912,0,0,0,13007,1122,0,not default
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2388,180000,male,university,2,29,1,2,2,-2,-1,...,0,1378,978,0,0,0,1704,0,2690,default
2389,500000,male,graduate school,2,32,1,-1,-1,-1,0,...,155951,159749,167416,1000,1267,155000,6000,10000,8000,not default
2390,50000,male,high school,1,32,2,3,2,2,2,...,41526,42209,44960,1500,1600,1700,1500,3600,0,default
2391,20000,female,high school,2,49,0,0,2,0,-1,...,10400,1150,0,3008,0,600,1150,0,0,not default


In [33]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

#make a copy of the original DataFrame
df_ml = df.copy()

#encode categorical variables
df_ml['SEX'] = df_ml['SEX'].astype('category').cat.codes
df_ml['EDUCATION'] = df_ml['EDUCATION'].astype('category').cat.codes
df_ml['MARRIAGE'] = df_ml['MARRIAGE'].astype('category').cat.codes
df_ml['DEFAULT PAYMENT NEXT MONTH (Y)'] = df_ml['DEFAULT PAYMENT NEXT MONTH (Y)'].map({'not default': 0, 'default': 1})

#define predictors and target variable
X = df_ml.drop(['DEFAULT PAYMENT NEXT MONTH (Y)'], axis=1)
y = df_ml['DEFAULT PAYMENT NEXT MONTH (Y)']

#split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

#train the Random Forest model
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

#predict on the entire original dataset
y_pred_full = model.predict(X)

#create a DataFrame with records predicted as fraud/suspicious
df_suspicious = df[y_pred_full == 1]

#show the first suspicious records
df_suspicious

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,DEFAULT PAYMENT NEXT MONTH (Y)
0,120000,female,university,2,26,-1,2,0,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,default
12,70000,male,university,2,30,1,2,2,0,0,...,66782,36137,36894,3200,0,3000,3000,1500,0,default
15,20000,male,graduate school,2,24,0,0,2,2,2,...,18338,17905,19104,3200,0,1500,0,1650,0,default
21,70000,female,university,2,26,2,0,0,2,2,...,44006,46905,46012,2007,3582,0,3601,0,1820,default
22,450000,female,graduate school,1,40,-2,-2,-2,-2,-2,...,560,0,0,19428,1473,560,0,0,1128,default
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2373,40000,male,university,1,38,2,2,2,2,2,...,33917,36704,40404,1300,2000,0,3500,4471,0,default
2375,210000,male,graduate school,2,33,1,-2,-2,-2,-1,...,0,700,0,0,0,0,700,0,0,default
2376,60000,male,university,2,30,0,0,0,2,2,...,17919,19167,18750,2000,2100,0,1500,0,1500,default
2388,180000,male,university,2,29,1,2,2,-2,-1,...,0,1378,978,0,0,0,1704,0,2690,default
